### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/medical_image_processing/

Mounted at /content/gdrive
/content/gdrive/MyDrive/medical_image_processing


In [ ]:
from fundal_image.utils import create_dir, get_filenames_sorted
from patches import generate_patches_images_only, reconstruct_patches
from metrics import iou, dice_coef, dice_loss, DiceBCELoss, bce
from metrics import compute_metrics, save_results
from utils import read_image_train, read_mask_train

In [ ]:
image_sizes = {"HRF": (2336, 3504), "drive": (584, 565), "chase_db": (960, 999)}
patch_size = {"HRF": (768, 768),
              "drive":(256, 256)
              }

dataset_name = "drive"
patch_size = patch_size[dataset_name]
patch_height = patch_size[0]
patch_width = patch_size[1]

In [ ]:
loss_str = "dice_bce"
with_augmentation = False
stride=256
seed = 42
run_id = 1
test_percentage = 0.4
class_weight = 0.4
class_weight_str = str(class_weight).replace(".", "_")
aug_string = ""
if with_augmentation:
    aug_string = "_hvr"
test_percentage_str = str(test_percentage).replace(".", "_")
root_path = f"/content/datasets/{dataset_name}_{patch_height}_{stride}_{loss_str}_{class_weight_str}_{aug_string}_{test_percentage_str}_{run_id}/"

images_path_train = f"{root_path}/training/images/"
gt_path_train = f"{root_path}/training/1st_manual/"
images_path_test = f"{root_path}/test/images/"
gt_path_test = f"{root_path}/test/1st_manual/"

create_dir(images_path_train)
create_dir(gt_path_train)
create_dir(images_path_test)
create_dir(gt_path_test)

In [ ]:
def wbce(actual, predicted):
    y_pred = K.clip(predicted, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - actual) * K.log(1 - y_pred)
    term_1 = class_weight * actual * K.log(y_pred + K.epsilon())
    return -K.mean(term_0 + term_1, axis=0)

In [ ]:
root_path

'/content/datasets/drive_256_256_dice_loss_0_4__0_4_1/'

In [ ]:
images_path_train

'/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/images/'

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/drive/training/images/*.* {images_path_train}
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/drive/training/1st_manual/*.* {gt_path_train}

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/drive/test/images/*.* {images_path_test}
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/drive/test/1st_manual/*.* {gt_path_test}

In [ ]:
!ls /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/drive/training/images

21_training.tif  26_training.tif  31_training.tif  36_training.tif
22_training.tif  27_training.tif  32_training.tif  37_training.tif
23_training.tif  28_training.tif  33_training.tif  38_training.tif
24_training.tif  29_training.tif  34_training.tif  39_training.tif
25_training.tif  30_training.tif  35_training.tif  40_training.tif


In [ ]:
# images_files, gt_files = get_filenames_sorted(images_path, gt_path)
# print(len(images_files), len(gt_files))
# # train_test_split
# from sklearn.model_selection import train_test_split
# label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
# xtrain, xtest, ytrain, ytest = train_test_split(images_files, gt_files, stratify=label,  test_size=test_percentage, random_state=seed)

In [ ]:
xtrain, ytrain = get_filenames_sorted(images_path_train, gt_path_train)
xtest, ytest = get_filenames_sorted(images_path_test, gt_path_test)

In [ ]:
print(len(xtrain), len(xtest))
# print(label)

20 20


In [ ]:
xtrain[:5]

['/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/images/21_training.tif',
 '/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/images/22_training.tif',
 '/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/images/23_training.tif',
 '/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/images/24_training.tif',
 '/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/images/25_training.tif']

In [ ]:
print(len(ytrain), len(ytest))
ytrain[:5]

20 20


['/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/1st_manual/21_manual1.gif',
 '/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/1st_manual/22_manual1.gif',
 '/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/1st_manual/23_manual1.gif',
 '/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/1st_manual/24_manual1.gif',
 '/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//training/1st_manual/25_manual1.gif']

In [ ]:
from tqdm import tqdm
import pandas as pd

In [ ]:
num_pixels_to_add = [0, 0]
if (image_sizes[dataset_name][0] % patch_size[0]) > 0:
    num_pixels_to_add[0] = ((image_sizes["drive"][0] // patch_size[0]) + 1) * patch_size[0]
    num_pixels_to_add[1] = ((image_sizes["drive"][1] // patch_size[1]) + 1) * patch_size[1]

split_name = "split_2"
data_path = root_path
splits =["train","test"]


split = "train"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_train, metadata_train_df, gt_patches_dict_train = generate_patches_images_only(xtrain,
                                                                image_patches_path,
                                                                patch_size=patch_size,
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytrain,
                                                                save_path_gt=gt_patches_path)
metadata_train_df.to_csv(f"{data_path}/{split_name}/image_patches_{stride}_{stride}_{split}.csv", index=False)



Generating patches for 20 images


100%|██████████| 20/20 [00:00<00:00, 24.92it/s]


In [ ]:
split = "test"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_test, metadata_test_df, gt_patches_dict_test = generate_patches_images_only(xtest,
                                                                image_patches_path,
                                                                patch_size=patch_size,
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytest,
                                                                save_path_gt=gt_patches_path)
metadata_test_df.to_csv(f"{data_path}/{split_name}/image_patches_{patch_height}_{stride}_{split}.csv", index=False)


Generating patches for 20 images


100%|██████████| 20/20 [00:00<00:00, 27.27it/s]


In [ ]:
metadata_test_df.shape

(180, 7)

In [ ]:
metadata_train_df.shape

(180, 7)

In [ ]:
metadata_train_df.head()

,image_filename,row,col,patch_height,patch_width,stride,num_fg_pixels
0,/content/datasets/drive_256_256_dice_loss_0_4_...,0,0,256,256,256,24179
1,/content/datasets/drive_256_256_dice_loss_0_4_...,0,256,256,256,256,39808
2,/content/datasets/drive_256_256_dice_loss_0_4_...,0,512,256,256,256,24058
3,/content/datasets/drive_256_256_dice_loss_0_4_...,256,0,256,256,256,39680
4,/content/datasets/drive_256_256_dice_loss_0_4_...,256,256,256,256,256,65536


In [ ]:
import glob
split = "test"
image_patches_path = f"{data_path}/{split_name}/{split}/image/"

reconstructed_original_path = f"{data_path}/{split_name}/reconstructed/{split}/image"
create_dir(reconstructed_original_path)
reconstruct_patches(im_patches_dict_test,
                    metadata_test_df,
                    reconstructed_original_path
                    )


In [ ]:
print(list(gt_patches_dict_test.keys()))

['01_test', '02_test', '03_test', '04_test', '05_test', '06_test', '07_test', '08_test', '09_test', '10_test', '11_test', '12_test', '13_test', '14_test', '15_test', '16_test', '17_test', '18_test', '19_test', '20_test']


In [ ]:
type(gt_patches_dict_test["01_test"])

numpy.ndarray

In [ ]:
reconstructed_gt_path = f"{data_path}/{split_name}/reconstructed/{split}/mask"

create_dir(reconstructed_gt_path)
reconstruct_patches(gt_patches_dict_test,
                    metadata_test_df,
                    reconstructed_gt_path,
                    )


In [ ]:
print([k for k in im_patches_dict_test.keys()])

['01_test', '02_test', '03_test', '04_test', '05_test', '06_test', '07_test', '08_test', '09_test', '10_test', '11_test', '12_test', '13_test', '14_test', '15_test', '16_test', '17_test', '18_test', '19_test', '20_test']


### Construct the Model:

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

### Adding the Metrics:

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def DiceBCELoss(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(inputs * targets)
    dice_loss = 1 - (2.*intersection + smooth)/(tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets) + dice_loss

    return Dice_BCE

def bce(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets)

    return Dice_BCE



### Training the Model:

In [ ]:
if loss_str == "bce":
    loss = bce
elif loss_str == "dice_bce":
    loss = DiceBCELoss
elif loss_str == "dice_loss":
    loss = dice_loss
elif loss_str == "wbce":
    loss = wbce
else:
    raise Exception("Invalid value for loss_str")
    
print(loss)

<function dice_loss at 0x7fbcf0572d40>


In [ ]:
H = patch_height
W = patch_width

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
from keras.metrics import AUC

In [ ]:
model_dir = f"{root_path}/trained_models/"

In [ ]:
create_dir(model_dir)

In [ ]:
!ls {model_dir}

In [ ]:
%cd /content/datasets

/content/datasets


In [ ]:
from glob import glob

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir(model_dir)

""" Hyperparameters """
batch_size = 8
lr = 1e-4
num_epochs = 50
model_path = os.path.join(f"{model_dir}", "model.h5")
csv_path = os.path.join(f"{model_dir}", "data.csv")

""" Dataset """
dataset_path = f"{data_path}/{split_name}"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
# train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
model.compile(loss=loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision(), AUC()])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 180 - 180
Valid: 180 - 180
Epoch 1/50
23/23 [==============================] - ETA: 0s - loss: 1.3206 - dice_coef: 0.1555 - iou: 0.0848 - recall: 0.7265 - precision: 0.1838 - auc: 0.8676
Epoch 1: val_loss improved from inf to 1.62412, saving model to /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//trained_models/model.h5
23/23 [==============================] - 23s 313ms/step - loss: 1.3206 - dice_coef: 0.1555 - iou: 0.0848 - recall: 0.7265 - precision: 0.1838 - auc: 0.8676 - val_loss: 1.6241 - val_dice_coef: 0.0886 - val_iou: 0.0464 - val_recall: 0.9952 - val_precision: 0.0489 - val_auc: 0.2752 - lr: 1.0000e-04
Epoch 2/50
23/23 [==============================] - ETA: 0s - loss: 1.0242 - dice_coef: 0.2487 - iou: 0.1428 - recall: 0.7576 - precision: 0.4575 - auc: 0.9473
Epoch 2: val_loss did not improve from 1.62412
23/23 [==============================] - 4s 185ms/step - loss: 1.0242 - dice_coef: 0.2487 - iou: 0.1428 - recall: 0.7576 - precision: 0.4575 - auc: 0.9473 - val

### Evaluating the Model:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.models import load_model

In [ ]:

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
import glob
supported_extensions = ["png", "jpg", "tif", "jpeg", "gif", "bmp"]

def image_files(path, file_prefix=None):
    all_image_files = []
    # print("path", path)
    # print("file_prefix", file_prefix)
    if file_prefix is None:
        file_names = list(glob.glob(path + "/*.*" ))
    else:
        prefix___ = path + "/" + file_prefix + ".*"
        #print("prefix", prefix___)
        file_names = list(glob.glob(prefix___))

    # print("filenames", file_names)

    for filename in file_names:
        ext = filename.rsplit(".", 1)[1]
        if ext.lower() in supported_extensions:
            all_image_files.append(filename)
    # print("All Image files", all_image_files)
    return sorted(all_image_files)


In [ ]:
results_path = f"{root_path}/results/"
prediction_path = f"{root_path}/predictions/"
prediction_np_path = f"{root_path}/predictions_np/"
create_dir(prediction_path)
create_dir(prediction_np_path)
create_dir(results_path)

In [ ]:
results_path

'/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//results/'

In [ ]:
from glob import glob

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Save the results in this folder """
# create_dir("results_sunil_test")
# create_dir(prediction_path)

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")

model = load_model(model_path, compile=False)

""" Load the dataset """
dataset_path = os.path.join(f"{data_path}/{split_name}", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    np.save(f"{prediction_np_path}/{name}.npy", y_pred)
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
    y_pred = np.squeeze(y_pred, axis=-1)

    """ Saving the images """
    save_image_path = f"{results_path}/{name}.png"
#    print(ori_x.shape, ori_y.shape, y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)
    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)
    dice_value = dice_coef(y.astype(np.float64), y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")
print(f"Dice Coeff: {score[6]:0.5f}")
""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coeff"])
df.to_csv(f"{results_path}/score.csv")

  0%|          | 0/180 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  1%|          | 1/180 [00:01<03:02,  1.02s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to rev

Accuracy: 0.97947
F1: 0.75070
Jaccard: 0.61400
Recall: 0.72675
Precision: 0.80298
AUC: 0.96830
Dice Coeff: 0.62750


# Reconstruct original image

In [ ]:
prediction_np_path

'/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/'

In [ ]:
from glob import glob

In [ ]:
# Create patches dictionary
predicted_patches_dict = dict()
for file in glob(f"{prediction_np_path}/*.npy"):
    print("file", file)
    patch = np.load(file)
    fname_without_ext = file.rsplit("/", 1)[1].rsplit(".", 1)[0]
    fname_without_ext = fname_without_ext[1:]
    patch_number = int(fname_without_ext.rsplit("_", 1)[1])
    image_file = fname_without_ext.rsplit("_", 1)[0]
    print(image_file, patch_number )

    if image_file in predicted_patches_dict.keys():
        predicted_patches_dict[image_file][patch_number] = np.load(file)
    else:
        patches = [None] * 140
        patches[patch_number] = np.load(file)
        predicted_patches_dict[image_file] = patches

file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_20_test_7.npy
20_test 7
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_17_test_8.npy
17_test 8
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_03_test_5.npy
03_test 5
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_03_test_0.npy
03_test 0
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_15_test_8.npy
15_test 8
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_13_test_7.npy
13_test 7
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_20_test_6.npy
20_test 6
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_11_test_4.npy
11_test 4
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_01_test_7.npy
01_test 7
file /content/datasets/drive_256_256_dice_loss_0_4__0_4_1//predictions_np/_14_test_1.npy
14_test 1
file /cont

In [ ]:
reconstructed_predictions_path = f"{data_path}/{split_name}/reconstructed/{split}/predictions_reconstructed"
create_dir(reconstructed_predictions_path)
reconstruct_patches(predicted_patches_dict,
                    metadata_test_df,
                    reconstructed_predictions_path,
                    mfactor=255
                    )


In [ ]:
!pwd

/content/datasets


In [ ]:
def read_mask(path):
    print(path)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
#    x = x.astype(np.int32)
    return ori_x, x

In [ ]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((ori_x.shape[0], 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    print(f"Saving image of size {cat_images.shape} to {save_image_path}")
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
results_path_reconstructed = f"{root_path}/results_reconstructed/"
create_dir(results_path_reconstructed)

In [ ]:
from glob import glob

In [ ]:
dataset_path = f"{root_path}/{split_name}/{split}/"
df_0_4, score_0_4 = compute_metrics(dataset_path,
                    results_path=results_path,
                    threshold=0.4,
                    model=model,
                    predictions_path=prediction_path,
                    prediction_np_path=prediction_np_path)

Threshold 0.4


  0%|          | 0/180 [00:00<?, ?it/s]

_01_test_0
(256, 256, 1) 1.0 0.0025882502
(256, 256, 3) (256, 256) (256, 256)


  1%|          | 1/180 [00:00<00:30,  5.96it/s]

_01_test_1


...

_20_test_7


 99%|█████████▉| 179/180 [00:30<00:00,  5.87it/s]

(256, 256, 1) 1.0 0.002548223
(256, 256, 3) (256, 256) (256, 256)
_20_test_8
(256, 256, 1) 0.9999999 0.0034619276
(256, 256, 3) (256, 256) (256, 256)


100%|██████████| 180/180 [00:30<00:00,  5.92it/s]

Threshold: 0.40000
Accuracy: 0.97921
F1: 0.75251
Jaccard: 0.61593
Recall: 0.74011
Precision: 0.79041
AUC: 0.96830
Dice Coeff: 0.62750


In [ ]:
dataset_path = f"{root_path}/{split_name}/{split}/"
df_0_5, score_0_5 = compute_metrics(dataset_path,
                    results_path=results_path,
                    threshold=0.5,
                    model=model,
                    predictions_path=prediction_path,
                    prediction_np_path=prediction_np_path)

Threshold 0.5


  0%|          | 0/180 [00:00<?, ?it/s]

_01_test_0
(256, 256, 1) 1.0 0.0025882502
(256, 256, 3) (256, 256) (256, 256)


  1%|          | 1/180 [00:00<00:29,  6.11it/s]

_01_test_1


...

_20_test_7


 99%|█████████▉| 179/180 [00:30<00:00,  5.99it/s]

(256, 256, 1) 1.0 0.002548223
(256, 256, 3) (256, 256) (256, 256)
_20_test_8
(256, 256, 1) 0.9999999 0.0034619276
(256, 256, 3) (256, 256) (256, 256)


100%|██████████| 180/180 [00:30<00:00,  5.90it/s]

Threshold: 0.50000
Accuracy: 0.97947
F1: 0.75070
Jaccard: 0.61400
Recall: 0.72675
Precision: 0.80298
AUC: 0.96830
Dice Coeff: 0.62750


In [ ]:
dataset_path = f"{root_path}/{split_name}/{split}/"
df_0_6, score_0_6 = compute_metrics(dataset_path,
                    results_path=results_path,
                    threshold=0.6,
                    model=model,
                    predictions_path=prediction_path,
                    prediction_np_path=prediction_np_path)

Threshold 0.6


  0%|          | 0/180 [00:00<?, ?it/s]

_01_test_0
(256, 256, 1) 1.0 0.0025882502
(256, 256, 3) (256, 256) (256, 256)


  1%|          | 1/180 [00:00<00:29,  5.99it/s]

_01_test_1


...

_20_test_7


 99%|█████████▉| 179/180 [00:30<00:00,  6.05it/s]

(256, 256, 1) 1.0 0.002548223
(256, 256, 3) (256, 256) (256, 256)
_20_test_8
(256, 256, 1) 0.9999999 0.0034619276
(256, 256, 3) (256, 256) (256, 256)


100%|██████████| 180/180 [00:30<00:00,  5.95it/s]

Threshold: 0.60000
Accuracy: 0.97962
F1: 0.74769
Jaccard: 0.61079
Recall: 0.71312
Precision: 0.81438
AUC: 0.96830
Dice Coeff: 0.62750


In [ ]:
results_path_reconstructed

'/content/datasets/drive_256_256_dice_loss_0_4__0_4_1//results_reconstructed/'

In [ ]:
!ls 

drive_256_256_dice_loss_0_4__0_4_1  logs


In [ ]:
folder_name = root_path.split("/")[3]
print(folder_name)

drive_256_256_dice_loss_0_4__0_4_1


In [ ]:
!zip -r {folder_name}.zip  {folder_name}/

  adding: drive_256_256_dice_loss_0_4__0_4_1/ (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/ (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/ (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/31_manual1.gif (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/22_manual1.gif (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/23_manual1.gif (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/24_manual1.gif (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/32_manual1.gif (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/40_manual1.gif (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/27_manual1.gif (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/34_manual1.gif (stored 0%)
  adding: drive_256_256_dice_loss_0_4__0_4_1/training/1st_man

In [ ]:
!ls -ltrh


total 430M
drwxr-xr-x  4 root root 4.0K Feb 24 07:06 logs
drwxr-xr-x 10 root root 4.0K Feb 24 07:11 drive_256_256_dice_loss_0_4__0_4_1
-rw-r--r--  1 root root 430M Feb 24 07:13 drive_256_256_dice_loss_0_4__0_4_1.zip


In [ ]:
!ls /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

02_g_test.png
drive_128_128_dice_bce_0_4__0_4_1
drive_128_128_dice_bce_0_4__0_4_1.zip
drive_128_128_dice_loss_0_4__0_4_1
drive_128_128_dice_loss_0_4__0_4_1.zip
drive_256_256_dice_bce_0_4__0_4_1
drive_256_256_dice_bce_0_4__0_4_1.zip
drive_64_64_dice_bce_0_4__0_4_1
drive_64_64_dice_bce_0_4__0_4_1.zip
drive_64_64_dice_loss_0_4__0_4_1
drive_64_64_dice_loss_0_4__0_4_1.zip
hrf_256_256_dice_loss_1
hrf_256_256_dice_loss_1.zip
hrf_256_256_dice_loss_hvr_1.zip
hrf_256_256_dice_loss_hvr_2.zip
hrf_512_256_bce_hvr_xval_10_42.zip
hrf_512_512
hrf_512_512_bce
hrf_512_512_bce_1
hrf_512_512_bce_1.zip
hrf_512_512_bce_2
hrf_512_512_bce_2.zip
hrf_512_512_bce_hvr
hrf_512_512_bce_hvr_0_3
hrf_512_512_bce_hvr_0_33_42.zip
hrf_512_512_bce_hvr_0_33_43
hrf_512_512_bce_hvr_0_33_43.zip
hrf_512_512_bce_hvr_0_3_44
hrf_512_512_bce_hvr_0_3_44.zip
hrf_512_512_bce_hvr_feb_3_1.zip
hrf_512_512_bce_hvr_feb_5_1.zip
hrf_512_512_bce_hvr_xval_2_42.zip
hrf_512_512_bce_hvr_xval_5_42
hrf_512_512_bce_hvr_xval_5_42_10epochs
hrf_512_51

In [ ]:
!cp {folder_name}.zip /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

In [ ]:
%cd /content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches/

/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches


In [ ]:
!pwd

/content/gdrive/MyDrive/Blood_Vessel_Results/hrf_patches


In [ ]:
!ls

02_g_test.png
drive_128_128_dice_bce_0_4__0_4_1
drive_128_128_dice_bce_0_4__0_4_1.zip
drive_128_128_dice_loss_0_4__0_4_1
drive_128_128_dice_loss_0_4__0_4_1.zip
drive_256_256_dice_bce_0_4__0_4_1
drive_256_256_dice_bce_0_4__0_4_1.zip
drive_256_256_dice_loss_0_4__0_4_1.zip
drive_64_64_dice_bce_0_4__0_4_1
drive_64_64_dice_bce_0_4__0_4_1.zip
drive_64_64_dice_loss_0_4__0_4_1
drive_64_64_dice_loss_0_4__0_4_1.zip
hrf_256_256_dice_loss_1
hrf_256_256_dice_loss_1.zip
hrf_256_256_dice_loss_hvr_1.zip
hrf_256_256_dice_loss_hvr_2.zip
hrf_512_256_bce_hvr_xval_10_42.zip
hrf_512_512
hrf_512_512_bce
hrf_512_512_bce_1
hrf_512_512_bce_1.zip
hrf_512_512_bce_2
hrf_512_512_bce_2.zip
hrf_512_512_bce_hvr
hrf_512_512_bce_hvr_0_3
hrf_512_512_bce_hvr_0_33_42.zip
hrf_512_512_bce_hvr_0_33_43
hrf_512_512_bce_hvr_0_33_43.zip
hrf_512_512_bce_hvr_0_3_44
hrf_512_512_bce_hvr_0_3_44.zip
hrf_512_512_bce_hvr_feb_3_1.zip
hrf_512_512_bce_hvr_feb_5_1.zip
hrf_512_512_bce_hvr_xval_2_42.zip
hrf_512_512_bce_hvr_xval_5_42
hrf_512_51

In [ ]:
!unzip {folder_name}.zip


Archive:  drive_256_256_dice_loss_0_4__0_4_1.zip
   creating: drive_256_256_dice_loss_0_4__0_4_1/
   creating: drive_256_256_dice_loss_0_4__0_4_1/training/
   creating: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/31_manual1.gif  
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/22_manual1.gif  
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/23_manual1.gif  
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/24_manual1.gif  
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/32_manual1.gif  
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/40_manual1.gif  
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/27_manual1.gif  
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/34_manual1.gif  
 extracting: drive_256_256_dice_loss_0_4__0_4_1/training/1st_manual/38_manual1.gif  
 extractin

In [ ]:
drive.flush_and_unmount()